In [1]:
import pandas as pd


class Film:
    def __init__(self, movie_id, imdb_id, title, release_year, genres):
        self.movie_id = movie_id
        self.imdb_id = imdb_id
        self.title = title
        self.release_year = release_year
        self.genres = genres.split("|")


df_movies = pd.read_csv("../Data/raw/movies.csv")
df_links = pd.read_csv("../Data/raw/links.csv")

merged_df = pd.merge(df_movies, df_links, on="movieId")
merged_df.drop(columns=["tmdbId"], inplace=True)

# Extract the year from the title and create a new column
merged_df["year"] = merged_df["title"].str.extract(r"\((\d{4})\)")

# Remove the year from the title
merged_df["title"] = merged_df["title"].str.replace(r"\s*\(\d{4}\)\s*", "", regex=True)

print(merged_df.sample(5))

       movieId                               title              genres  \
81553   274009                         Night's End              Horror   
51201   181397                       Mystification  (no genres listed)   
86568   289019                         Under Siege     Action|Thriller   
29821   134485                        Sturmtruppen  (no genres listed)   
86250   288033  Under the Sea: A Descendants Story             Fantasy   

         imdbId  year  
81553  14870640  2022  
51201   1630645  2010  
86568     80710  1980  
29821     76777  1976  
86250   8884364  2018  


In [ ]:
subset_df = merged_df.sample(5)

films_list = []

for index, row in subset_df.iterrows():
    film = Film(row["movieId"], row["imdbId"], row["title"], row["year"], row["genres"])
    films_list.append(film)

for film in films_list:
    print(f"Title: {film.title}, Year: {film.release_year}, Genres: {film.genres}")

Title: The Case of the Lucky Legs, Year: 1935, Genres: ['Comedy', 'Crime', 'Mystery']
Title: The Most Beautiful Day, Year: 2016, Genres: ['Comedy', 'Drama']
Title: Snow-White, Year: 1933, Genres: ['(no genres listed)']
Title: Quigley Down Under, Year: 1990, Genres: ['Adventure', 'Drama', 'Western']
Title: The Creative Brain, Year: 2019, Genres: ['Documentary']
